In [1]:
using Revise

In [2]:
using Flux
using JLD2
using FileIO
using MLDataPattern
using CoordinateTransformations
using ProgressMeter
using RigidBodyDynamics
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
import FluxExtensions
import LearningMPC
import LCPSim
import Hoppers

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/JLD2.ji for module JLD2.
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/MLDataPattern.ji for module MLDataPattern.
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/TypeSortedCollections.ji for module TypeSortedCollections.
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/RigidBodyDynamics.ji for module RigidBodyDynamics.
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/Hoppers.ji for module Hoppers.


In [6]:
samples = load("2018-01-30-hopper-grid/grid_search.jld2")["samples"];

In [7]:
filter!(samples) do sample
    sample.state[3] <= sample.state[1]
end;

In [8]:
features(sample::LearningMPC.Sample) = (sample.state, sample.uJ[:, 1])
data = features.(samples);
train_data, test_data = splitobs(shuffleobs(data), at=0.85);

In [47]:
function setup_model()
    signals = Chain(Dense(4, 2 * 16), x -> reshape(x, 16, 2))
    weights = Chain(
        LinearMap(UniformScaling(0.2)),
        Dense(4, 16, elu),
        Dense(16, 16, elu),
        softmax)
    model = FluxExtensions.Attention(signals, weights)
            
    loss = (x, y) -> Flux.mse(model(x), y)
    model, loss
end

model, loss = setup_model()
opt = Flux.ADADelta(params(model))

(::#71) (generic function with 1 method)

In [48]:
model(ones(4))

Tracked 1×2 Array{Float64,2}:
 0.0341821  -0.176262

In [51]:
import LCPSim
import Hoppers
using RigidBodyDynamics
using Gurobi
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

true

In [53]:
robot = Hoppers.Hopper()
xstar = Hoppers.nominal_state(robot)
ustar = zeros(num_velocities(xstar))
basevis = Visualizer()[:hopper]
setgeometry!(basevis, robot)
settransform!(basevis[:robot], xstar)
Δt = 0.05

0.05

In [55]:
net_controller = x -> vec(Flux.Tracker.value(model(state_vector(x))))

(::#33) (generic function with 1 method)

In [59]:
x_init = MechanismState{Float64}(robot.mechanism)
set_configuration!(x_init, [1.0, 1.0])
set_velocity!(x_init, [0., 0.])
LearningMPC.randomize!(x_init, x_init, 0.5, 1.0)
results = LCPSim.simulate(x_init, net_controller,
    robot.environment,
    Δt,
    100,
    GurobiSolver(Gurobi.Env(), OutputFlag=0));

In [60]:
LearningMPC.playback(basevis[:robot], results, Δt)

In [79]:
@showprogress for i in 1:1000
    Flux.train!(loss, train_data, opt)
end

Progress:  26%|███████████                              |  ETA: 1:23:56

LoadError: [91mLoss is NaN[39m

In [80]:
params(model)

6-element Array{Any,1}:
 param([-14.7046 -9.47962 22.265 -5.13429; -14.8604 -9.06943 22.4948 -5.60649; … ; 0.962631 -3.55739 -4.20996 0.229343; -14.3882 -9.81874 21.9935 -4.59536])                                                
 param([-6.06916, -6.11781, -6.2637, -6.09274, -6.5911, -6.3643, -6.40997, -3.11137, 3.34318, -6.46791  …  -6.26744, -3.47873, 3.34407, -6.34105, 17.8969, -5.95965, -6.01945, -6.22724, 2.70026, -6.12001])
 param([NaN NaN NaN NaN; NaN NaN NaN NaN; … ; NaN NaN NaN NaN; NaN NaN NaN NaN])                                                                                                                            
 param([NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN])                                                                                                                    
 param([NaN NaN … NaN NaN; NaN NaN … NaN NaN; … ; NaN NaN … NaN NaN; NaN NaN … NaN NaN])                                                                    

In [67]:
x_init = MechanismState{Float64}(robot.mechanism)
set_configuration!(x_init, [1.0, 1.0])
set_velocity!(x_init, [0., 0.])
LearningMPC.randomize!(x_init, x_init, 0.5, 1.0)
results = LCPSim.simulate(x_init, net_controller,
    robot.environment,
    Δt,
    100,
    GurobiSolver(Gurobi.Env(), OutputFlag=0));

In [68]:
LearningMPC.playback(basevis[:robot], results, Δt)

In [75]:
nn_controller = x -> begin
    v = state_vector(x)
    i = indmin((norm(xi .- v) for (xi, ui) in train_data))
    train_data[i][2]
end

(::#35) (generic function with 1 method)

In [77]:
x_init = MechanismState{Float64}(robot.mechanism)
set_configuration!(x_init, [1.0, 1.0])
set_velocity!(x_init, [0., 0.])
LearningMPC.randomize!(x_init, x_init, 0.5, 1.0)
results = LCPSim.simulate(x_init, nn_controller,
    robot.environment,
    Δt,
    100,
    GurobiSolver(Gurobi.Env(), OutputFlag=0));

In [78]:
LearningMPC.playback(basevis[:robot], results, Δt)